# Código One Class Classification

## Imports

## Funções de Suporte

## K-distance

# Testes

## Base de Dados 1 -

### Preparação

### Execução

### Resultados

## Base de Dados 2 -

### Preparação

### Execução

### Resultados